# Fine-tuning LLMs for News Category

Inspired by [Kshitiz Sahay's blog](https://medium.com/@kshitiz.sahay26/fine-tuning-llama-2-for-news-category-prediction-a-step-by-step-comprehensive-guide-to-fine-tuning-48c06dee28a9)

step-by-step tutorial for fine-tuning any LLM (Large Language Model). 

This guide will be divided into two parts:

**Part 1: Setting up and Preparing for Fine-Tuning**
1. Installing and loading the required modules
2. Steps to get approval for Meta's Llama 2 family of models
3. Setting up Hugging Face CLI and user authentication
4. Loading a pre-trained model and its associated tokenizer
5. Loading the training dataset
6. Preprocessing the training dataset for model fine-tuning

**Part 2: Fine-Tuning and Open-Sourcing**
1. Configuring PEFT (Parameter Efficient Fine-Tuning) method QLoRA for efficient fine-tuning
2. Fine-tuning the pre-trained model
3. Saving the fine-tuned model and its associated tokenizer


### Installing Required Libraries

First, we will install some required libraries.

`transformers`: for loading a large language model and fine-tuning it.

`bitsandbytes`: for loading the model in 4-bit precision.

`accelerate`: for training models and performing inference at scale.

`peft`: for fine-tuning a small number of parameters.

`trl`: for training transformer language models using Reinforcement Learning.


In [ ]:
!pip install -q accelerate==0.30.1 --progress-bar off
!pip install -q peft==0.11.1 --progress-bar off
!pip install -q bitsandbytes==0.43.1 --progress-bar off
!pip install -q transformers==4.40.0 --progress-bar off
!pip install -q trl==0.9.6 --progress-bar off
# torch 2.1.0+cu122

### Loading Required Libraries

Next, we will load the required libraries for fine-tuning a Large Language Model (LLM) like Llama 2. We will look at each imported class in greater detail in subsequent sections.

In [2]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

Setting ds_accelerator to cuda (auto detect)


### Hugging Face Hub Login
See related docs

openlm-research/open_llama_3b_v2 is used in this doc

In [4]:
#!huggingface-cli login

import os
import torch

os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ['WANDB_NOTEBOOK_NAME'] = "NewsClassification"

torch.cuda.empty_cache()

BASE_DIR = '/mlx_devbox/users/haidong.shao/playground/'
#model_name = "openlm-research/open_llama_3b_v2"
model_name = "Qwen/Qwen2-0.5B"

### Creating Bitsandbytes Configuration

Before loading the model, we will define a function `create_bnb_config` to define the `bitsandbytes` configuration. The `bitsandbytes` library allows model quantization. Quantization is a technique used to compress deep learning models by reducing the number of bits used to represent their weights and activations. This compression allows for faster inference and reduced memory consumption, making it possible to deploy these models on edge devices with limited resources.

By using 4-bit transformer language models, we can achieve impressive results while significantly reducing memory and computational requirements.

Hugging Face Transformers (`transformers`) is closely integrated with `bitsandbytes`. The `BitsAndBytesConfig` class from the `transformers` library allows configuring the model quantization method.

Parameters:

`load_in_4bit`: Load the model in 4-bit precision, i.e., divide memory usage by 4.

`bnb_4bit_use_double_quant`: Use nested quantization techniques for more memory-efficient inference at no additional cost.

`bnb_4bit_quant_type`: Set quantization data type. The options are either FP4 (4-bit precision), which is the default quantization data type, or NF4 (Normal Float 4), a new 4-bit data type adapted for weights that have been initialized using a normal distribution.

`bnb_4bit_compute_dtype`: Set the computational data type for 4-bit models. Default value: torch.float32

In [5]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

### Loading Hugging Face Model and Tokenizer

We will now define a function `load_model` that accepts the model name (`model_name`) from Hugging Face Hub and the `bitsandbytes` configuration for model quantization.

In this function, we will perform the following steps:
 1. Get the number of GPUs available.
 2. Set the maximum GPU memory.
 3. Use the from_pretrained` method from the `AutoModelForCausalLM` class to load a pre-trained Hugging Face model in 4-bit precision using the model name and the quantization configuration.
 4. Set which device to send the model to using `device_map`. Passing `device_map = 0` means putting the whole model on GPU 0. Other inputs could be `cpu`, `cuda:1`, etc. Setting `device_map = auto` will let `accelerate` compute the most optimized `device_map` automatically.
 5. Set `max_memory`, a dictionary device identifier, to maximum memory, which will default to the maximum memory available for each GPU and the available CPU RAM if unset.
 6. Load the model tokenizer from the model name on Hugging Face.
 7. Set a padding token to ensure shorter sequences will have the same length as the longest sequence in a batch. In this case, we will set the EOS (End of Sentence) token as the padding token.

 **Important Note:  A tokenizer for a model will preprocess and tokenize (convert letters/words/sub-words to tokens or numbers) the input in a way that the model expects. Model tokenizers are also responsible for correctly applying special tokens and certain special embeddings or positional encoders specific to a model in the input.**

In [6]:
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{32500}MB'

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

### Initializing Transformers and Bitsandbytes Parameters

We will now initialize input parameters for the `transformers` and `bitsandbytes` modules.

In [7]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

Finally, we will call the above functions to get `model` and `tokenizer` objects.

In [8]:
# Load model from Hugging Face Hub with model name and bitsandbytes configuration

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(model_name, bnb_config)

/usr/local/lib/python3.9/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Loading Dataset

Now that we have loaded the Llama-2-7B model and its tokenizer, we will move on to loading our news classification instruction dataset from the previous blog as a Hugging Face `Datasets`.

Firstly, we will initialize the path of the dataset. In this case, we have a CSV file that contains 99 records, or prompts. This dataset contains an `instruction` column containing the instruction to categorize a news article into 18 categories, an `input` column containing the news article, and an `output` column containing the actual news category for training.

We will use the `load_dataset` function and pass the file location. We will define a generic dataset builder name `csv` because our dataset is a CSV file. You can similarly load a JSON file by passing `json` and the dataset location to a JSON file. All the records are assigned to the `train` split by default, which we would retrieve using the `split` parameter.

In [9]:
#Generate training data

#import pandas as pd

#BASE_DIR = '/mlx_devbox/users/haidong.shao/playground/'
#dataset_name = "bbc-text.csv"
#df = pd.read_csv(BASE_DIR+dataset_name)
#instruction_value = 'Categorize the news article into one of the 5 categories:\n\n' \
#                    'tech\nbusiness\nsport\nentertainment\npolitics\n\n'
#df['instruction'] = instruction_value

#df = df.rename(columns={'text': 'input'})
#df = df.rename(columns={'category': 'output'})
#df = df[['instruction', 'input', 'output']]
#display(df), df.shape
#df['input'][0]
#df['output'][0]
#df['instruction'][0]

#df.to_csv(BASE_DIR+'processed_new.csv', index=False)

In [10]:
# Load dataset
dataset_name=BASE_DIR+'ft-lora/processed_new.csv'
dataset = load_dataset("csv", data_files = dataset_name, split = "train")


In [11]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')
#dataset[randrange(len(dataset))]

Number of prompts: 2225
Column names are: ['instruction', 'input', 'output']


The `load_dataset` function will convert the CSV file into a dictionary of prompts. We can look at a random prompt in the dataset using a random index.

In [12]:
dataset[randrange(len(dataset))]

{'instruction': 'Categorize the news article into one of the 5 categories:\n\ntech\nbusiness\nsport\nentertainment\npolitics\n\n',
 'input': 'greek pair attend drugs hearing greek sprinters kostas kenteris and katerina thanou have appeared before an independent tribunal which will decide if their bans should stand.  they were given provisional suspensions by athletics  ruling body the iaaf in december for failing to take drugs tests before the athens olympics. the pair arrived with former coach christos tzekos to give evidence at the hellenic olympic committee s offices. a decision is expected to be announced before the end of february. whatever the ruling  all parties will have the right to appeal to the court of arbitration for sport. yiannis papadoyiannakis  who was head of the greek olympic team at the athens games last year  also testified at the tribunal  along with other greek sports officials and athletes.  i believe the tribunal will reach a decision that will uphold the stand

### Creating Prompt Template

After loading the instruction dataset, we will define the `create_prompt_formats` function to create a prompt template against each prompt in our dataset and save it in a new dictionary key `text` for further data preprocessing and fine-tuning.

In [13]:
def create_prompt_formats(sample):
    """
    Creates a formatted prompt template for a prompt in the instruction dataset

    :param sample: Prompt or sample from the instruction dataset
    """

    # Initialize static strings for the prompt template
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"

    # Combine a prompt with the static strings
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['input']}" if sample["input"] else None
    response = f"{RESPONSE_KEY}\n{sample['output']}"
    end = f"{END_KEY}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [14]:
create_prompt_formats(dataset[randrange(len(dataset))])

{'instruction': 'Categorize the news article into one of the 5 categories:\n\ntech\nbusiness\nsport\nentertainment\npolitics\n\n',
 'input': 'sideways dominates spirit awards the comedy sideways has dominated this year s independent spirit awards  winning all six of the awards for which it was nominated.  it was named best film while alexander payne won best director and best screenplay  along with writing partner jim taylor. it also won acting awards for stars paul giamatti  thomas haden church and virginia madsen. sideways is tipped to do well at sunday s oscars  with five nominations.  the awards  now in their 20th year  are given to films made outside the traditional studio system  and are traditionally held the day before the oscars. other winners included catalina sandino moreno  who took best actress for her role as a drug smuggler in the colombian drama maria full of grace. moreno is also nominated for best actress at the oscars. the best first screenplay award went to joshua m

### Getting Maximum Sequence Length of the Pre-trained Model

In the next cell, we will define the `get_max_length` function to find out the maximum sequence length of the Llama-2-7B model. This function will pull the model configuration and attempt to find the maximum sequence length from one of the several configuration keys that may contain it. If the maximum sequence length is not found, it will default to 1024. We will use the maximum sequence length during dataset preprocessing to remove records that exceed that context length because the pre-trained model won't accept them.

In [15]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

### Tokenizing Dataset Batch

The user-defined `preprocess_batch` function will tokenize a batch of the input dataset (`batch`) using the `tokenizer` object. We will set the maximum sequence length using the `max_length` parameter, which will control the maximum length used by the padding or truncation parameter. `truncation = True` will truncate the input to the maximum length provided by the `max_length` parameter. Similarly, `padding = max_length` will pad the input to the maximum length provided. This function will be called in the `preprocess_dataset` function defined next.

In [16]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

### Preprocessing Dataset

To preprocess the complete dataset for fine-tuning, we will define the `preprocess_dataset` function, which will perform the following operations:

1. Create the formatted prompts against each prompt in the instruction dataset using the `create_prompt_formats` function.
2. Tokenize the dataset in batches using the `preprocess_batch` function and removing the original dictionary keys (instruction, input, output, and text).
3. Filter out prompts with input token sizes exceeding the maximum length.
4. Shuffle the dataset using a random seed.

In [17]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["instruction", "input", "output", "text"],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [18]:
# Random seed
seed = 33

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)
preprocessed_dataset = preprocessed_dataset.train_test_split(test_size=0.05, seed=42)

Found max lenth: 131072
Preprocessing dataset...


Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2225 [00:00<?, ? examples/s]

We can now look at the preprocessed dataset, which contains tokens or IDs.

In [19]:
print(preprocessed_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2113
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 112
    })
})


In [20]:
print(preprocessed_dataset['train'][0])

{'input_ids': [38214, 374, 458, 7600, 429, 16555, 264, 3383, 13, 9645, 264, 2033, 429, 34901, 44595, 279, 1681, 382, 14374, 29051, 510, 34, 7593, 551, 279, 3669, 4549, 1119, 825, 315, 279, 220, 20, 11059, 1447, 17785, 198, 26151, 198, 60901, 198, 306, 11205, 198, 9896, 46979, 1022, 2505, 510, 14277, 17672, 4243, 3437, 76441, 66510, 17672, 4243, 3437, 220, 279, 85919, 2114, 315, 19197, 792, 8644, 323, 4780, 220, 702, 1053, 432, 686, 4559, 220, 17, 20, 18, 76, 32488, 53815, 16, 22, 20, 76, 26, 400, 18, 17, 23, 76, 8, 315, 501, 13248, 438, 432, 5868, 311, 5648, 1640, 93930, 13, 220, 279, 6278, 374, 279, 1537, 949, 315, 264, 3119, 311, 312, 7837, 220, 17, 13, 19, 11081, 32488, 2630, 2364, 315, 44205, 13, 8818, 19962, 2474, 432, 572, 8930, 304, 220, 16, 24, 24, 17, 220, 17672, 4243, 3437, 702, 5926, 1865, 5098, 304, 13054, 1181, 2562, 2163, 323, 11727, 6625, 614, 12771, 705, 13, 4764, 220, 30399, 2058, 3405, 3425, 432, 60091, 3322, 15255, 311, 4717, 1787, 220, 1496, 448, 279, 67927, 13, 220

With everything set up, we can move forward to fine-tuning or instruction-tuning Llama-2-7B on our news classification instruction dataset.

### Creating PEFT Configuration


Fine-tuning pretrained LLMs on downstream datasets results in huge performance gains when compared to using the pretrained LLMs out-of-the-box. However, as models get larger and larger, full fine-tuning becomes infeasible to train on consumer hardware. In addition, storing and deploying fine-tuned models independently for each downstream task becomes very expensive, because fine-tuned models are the same size as the original pretrained model. Parameter-Efficient Fine-tuning (PEFT) approaches are meant to address both problems!


PEFT approaches only fine-tune a small number of (extra) model parameters while freezing most parameters of the pretrained LLMs, thereby greatly decreasing the computational and storage costs. It also helps in portability, wherein users can tune models using PEFT methods to get tiny checkpoints worth a few MB compared to the large checkpoints of full fine-tuning.


**In short, PEFT approaches enable you to get performance comparable to full fine-tuning while only having a small number of trainable parameters.**


Hugging Face provides the PEFT library, which provides the latest Parameter-Efficient Fine-tuning techniques seamlessly integrated with Hugging Face Transformers and Hugging Face Accelerate.


There are several PEFT methods. In the next cell, we will use QLoRA, one of the latest methods that reduces the memory usage of LLM finetuning without performance tradeoffs, using the `LoraConfig` class from the `peft` library.


QLoRA uses 4-bit quantization to compress a pretrained language model. The LM parameters are then frozen, and a relatively small number of trainable parameters are added to the model in the form of Low-Rank Adapters. During finetuning, QLoRA backpropagates gradients through the frozen 4-bit quantized pretrained language model into the Low-Rank Adapters. The LoRA layers are the only parameters being updated during training.

In [21]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

### Finding Modules for LoRA Application

In the next cell, we will define the `find_all_linear_names` function to find the module to apply LoRA to. This function will get the module names from `model.named_modules()` and store it in a set to keep distinct module names.

In [22]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

### Calculating Trainable Parameters

We can use the `print_trainable_parameters` function to find out the number and percentage of trainable model parameters. This function will calculate the number of total parameters in `model.named_parameters()` and then those that would get updated.

In [23]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

### Fine-tuning the Pre-trained Model

We will create `fine_tune`, our final function, to wrap everything we have done so far and initiate the fine-tuning process. This function will perform the following model preprocessing operations to prepare it for training:


1. Enable gradient checkpointing to reduce memory usage during fine-tuning.
2. Use the `prepare_model_for_kbit_training` function from PEFT to prepare the model for fine-tuning.
3. Call find_all_linear_names` to get the module names to apply LoRA to.
4. Create LoRA configuration by calling the `create_peft_config` function.
5. Wrap the base Hugging Face model for fine-tuning to PEFT using the `get_peft_model` function.
6. Print the trainable parameters.


For training, we will instantiate a `Trainer()` object within the `fine_tune` function. This class requires the model, preprocessed dataset, and training arguments, listed below.


`per_device_train_batch_size`: The batch size per GPU/TPU/CPU for training.


`gradient_accumulation_steps`: Number of update steps to accumulate the gradients for, before performing a backward/update pass.


`warmup_steps`: Number of steps used for a linear warmup from 0 to `learning_rate`.


`max_steps`: If set to a positive number, the total number of training steps to perform.


`learning_rate`: The initial learning rate for Adam.


`fp16`: Whether to use 16-bit (mixed) precision training (through NVIDIA apex) instead of 32-bit training.


`logging_steps`: Number of update steps between two logs.


`output_dir`: The output directory where the model predictions and checkpoints will be written.


`optim`: The optimizer to use for training.


Next, we will use the `train` method on the trainer` object to start the training and log and save the model metrics on the training dataset. Finally, we will save the model checkpoint (model weights, configuration file, and tokenizer) in the output directory and delete the model to free up memory. You can load the model for inference later using its saved checkpoint.

In [24]:
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
            train_dataset=dataset['train'],
            eval_dataset = dataset['test'],
            args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            save_strategy="steps",
            save_steps=50,
            evaluation_strategy="steps",
            eval_steps=25,
            optim = optim,
            gradient_checkpointing=True,  # Leads to reduction in memory at slighly decrease in speed
            gradient_checkpointing_kwargs={"use_reentrant": False},
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    # Save model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

Initializing QLoRA and TrainingArguments parameters below for training.

In [25]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

In [26]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = BASE_DIR+"results"

# Batch size per GPU for training
per_device_train_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 150

# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

Calling the `fine_tune` function below to fine-tune or instruction-tune the pre-trained model on our preprocessed news classification instruction dataset.

In [27]:
fine_tune(model,
      tokenizer,
      preprocessed_dataset,
      lora_r,
      lora_alpha,
      lora_dropout,
      bias,
      task_type,
      per_device_train_batch_size,
      gradient_accumulation_steps,
      warmup_steps,
      max_steps,
      learning_rate,
      fp16,
      logging_steps,
      output_dir,
      optim)

LoRA module names: ['o_proj', 'down_proj', 'q_proj', 'up_proj', 'gate_proj', 'k_proj', 'v_proj']
All Parameters: 323,917,696 || Trainable Parameters: 8,798,208 || Trainable Parameters %: 2.716186274676392


Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/usr/local/lib/python3.9/dist-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
max_steps is given, it will override any value given in num_train_epochs


Training...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/local/lib/python3.9/dist-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
wandb: ⭐️ View project at https://ml.byteintl.net/experiment/tracking/detail?Id=project_20230126_e9daa974
wandb: 🚀 View run at https://ml.byteintl.net/experiment/tracking/detail?Id=project_20230126_e9daa974&selectedTrial=run_20240708_36e400d1


Step,Training Loss,Validation Loss
25,2.278700,2.665203
50,2.428400,2.614667
75,2.521700,2.592487
100,2.477800,2.577690
125,2.183700,2.568871
150,2.312200,2.565486


...

***** train metrics *****
  epoch                    =      0.284
  total_flos               =   678026GF
  train_loss               =     2.5154
  train_runtime            = 0:05:56.28
  train_samples_per_second =      1.684
  train_steps_per_second   =      0.421
{'train_runtime': 356.2844, 'train_samples_per_second': 1.684, 'train_steps_per_second': 0.421, 'total_flos': 728025672660480.0, 'train_loss': 2.5154240051905314, 'epoch': 0.28395646000946523}
Saving last checkpoint of the model...


/usr/local/lib/python3.9/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f0854eccfa0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f0854423ee0, execution_count=27 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f0854423d90, raw_cell="fine_tune(model,
      tokenizer,
      preprocess.." store_history=True silent=False shell_futures=True cell_id=695a7650-8cbd-458b-b109-aa83cbdc1ad4> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

wandb: Waiting for W&B process to finish... (success).
wandb: - 0.011 MB of 0.011 MB uploaded (0.000 MB deduped)
wandb: Run history:
wandb:               eval/loss █▄▃▂▁▁
wandb:            eval/runtime █▁▁▁▁▁
wandb: eval/samples_per_second ▁█████
wandb:   eval/steps_per_second ▁█████
wandb:             train/epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:       train/global_step ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:         train/grad_norm ▄█▂▂▂▂▁▂▁▂▁▁▁▁▂▁▂▁▂▂▁▁▁▂▂▁▂▁▁▁▁▁▂▁▁▁▂▁▁▁
wandb:     train/learning_rate ████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:              train/loss █▅▄▂▄▂▃▂▃▅▂▃▃▂▃▂▁▃▄▂▄▂▃▁▂▂▃▃▄▃▂▆▃▂▂▄▂▁▄▁
wandb: 
wandb: Run summary:
wandb:                eval/loss 2.56549
wandb:             eval/runtime 6.9844
wandb:  eval/samples_per_second 16.036
wandb:    eval/steps_per_second 2.004
wandb:               total_flos 728025672660480.0
wandb:              train/epoch 0.28396
wandb:        train/global_step 150
wandb:          train/grad_norm 1.75214
wandb

With these steps, we have fine-tuned a popular open-source pre-trained model, on an instruction dataset that we created for news classification!

### Merging Weights & Pushing to Hugging Face

After saving the fine-tuned weights, we can create our fine-tuned model by merging the fine-tuned weights and saving it to a new directory with its tokenizer. By performing this step, we can have a memory-efficient, fine-tuned model and tokenizer for inference. We will also push the fine-tuned model and its associated tokenizer to Hugging Face Hub for public usage.


In [26]:
# Load fine-tuned weights
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map = "auto", torch_dtype = torch.bfloat16)
# Merge the LoRA layers with the base model
model = model.merge_and_unload()

# Save fine-tuned model at a new location
output_merged_dir = BASE_DIR+"results/news_classification_open-llama-3b-v2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)

# Save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fbb813e0ee0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fbb802f6910, raw_cell="# Load fine-tuned weights
model = AutoPeftModelFor.." store_history=True silent=False shell_futures=True cell_id=4a5a04d5-b7f3-40d9-b75e-399f9cba2bab>,),kwargs {}:


TypeError: _resume_backend() takes 1 positional argument but 2 were given

/usr/local/lib/python3.9/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/tokenizer_config.json',
 '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/special_tokens_map.json',
 '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/tokenizer.model',
 '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/added_tokens.json',
 '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/tokenizer.json')

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fbb813e0ee0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fbb802f6640, execution_count=26 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fbb802f6910, raw_cell="# Load fine-tuned weights
model = AutoPeftModelFor.." store_history=True silent=False shell_futures=True cell_id=4a5a04d5-b7f3-40d9-b75e-399f9cba2bab> result=('/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/tokenizer_config.json', '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/special_tokens_map.json', '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/tokenizer.model', '/mlx_devbox/users/haidong.shao/playground/results/news_classification_open-llama-3b-v2/final_merged_checkpoint/added_

TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [7]:
#model

In [32]:
tokenizer

LlamaTokenizerFast(name_or_path='openlm-research/open_llama_3b_v2', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)

In [8]:
# Push fine-tuned model and tokenizer to Hugging Face Hub
#new_model = "seastii/news-classification-open-llama-3b-v2"
#model.push_to_hub(new_model, use_auth_token = "")
#tokenizer.push_to_hub(new_model, use_auth_token = "")